<a href="https://colab.research.google.com/github/Guanhuchang/AGNet/blob/master/train1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import time

# from model import Model
# 定义训练的设备
device = torch.device("cuda")
train_dataset = torchvision.datasets.CIFAR10(
    root="data",
    train=True,
    transform=torchvision.transforms.ToTensor(),
    download=True
)
test_dataset = torchvision.datasets.CIFAR10(
    root="data",
    train=False,
    transform=transforms.ToTensor(),
    download=True
)
# length 长度
train_dataset_size = len(train_dataset)
test_dataset_size = len(test_dataset)
# 如果train_data_size=10,训练数据集的长度为：10
print("训练数据集的长度为：{}".format(train_dataset_size))
print("测试数据集的长度为：{}".format(test_dataset_size))
# 利用DataLoader来加载数据集
train_dataloader = DataLoader(train_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)


# 搭建神经网络
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.model1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, padding=2),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, padding=2),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, padding=2),
            nn.MaxPool2d(kernel_size=2),
            nn.Flatten(),
            nn.Linear(in_features=1024, out_features=64),
            nn.Linear(in_features=64, out_features=10)
        )

    def forward(self, x):
        x = self.model1(x)
        return x


model = Model()
model=model.to(device)
# 损失函数
loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.to(device)
# 优化器
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
# 设置训练网络的一些参数
# 记录训练的次数
total_train_step = 0
# 记录测试的次数
total_test_step = 0
# 记录训练的轮次
epoch = 10
# 添加tensorboard
writer = SummaryWriter("logs_train")
start_time = time.time()
for i in range(epoch):
    print("-----------第{}轮训练开始----------".format(i + 1))

    # 训练步骤开始
    model.train()  #
    for data in train_dataloader:
        imgs, targets = data
        imgs = imgs.to(device)
        targets = targets.to(device)
        outputs = model(imgs)
        loss = loss_fn(outputs, targets)

        # 优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step = total_train_step + 1
        if total_train_step % 100 == 0:
            end_time = time.time()
            print(end_time-start_time)
            print("训练次数：{}，loss:{}".format(total_train_step, loss.item()))
            writer.add_scalar("train_loss", loss.item(), total_train_step)

    # 测试集开始
    model.eval()
    total_test_loss = 0
    total_accurary = 0
    with torch.no_grad():  # 调用不调参
        for data in test_dataloader:
            imgs, targets = data
            imgs = imgs.to(device)
            targets = targets.to(device)
            outputs = model(imgs)
            loss = loss_fn(outputs, targets)
            total_test_loss = total_test_step + loss.item()
            accurary = (outputs.argmax(1) == targets).sum()
            total_accurary = total_accurary + accurary

    print(f"整体测试集上的Loss:{total_test_loss}")
    print("整体测试集上的正确率：{}".format(total_accurary / test_dataset_size))
    writer.add_scalar("test_loss", total_test_loss, total_test_step)
    writer.add_scalar("test_accuracy", total_accurary / test_dataset_size, total_test_step)
    total_test_step = total_test_step + 1

    torch.save(model, "model_{}.pth".format(i))
    # torch.save(model.state_dict(),"tudui_{}.pth".format(i))
    print("模型已保存")

writer.close()


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified
训练数据集的长度为：50000
测试数据集的长度为：10000
-----------第1轮训练开始----------
6.678032636642456
训练次数：100，loss:2.283416271209717
8.273031949996948
训练次数：200，loss:2.272122383117676
9.840346813201904
训练次数：300，loss:2.2170214653015137
11.397895812988281
训练次数：400，loss:2.0979089736938477
12.958790063858032
训练次数：500，loss:2.0328633785247803
14.522661924362183
训练次数：600，loss:1.9972784519195557
16.060158491134644
训练次数：700，loss:1.9855588674545288
整体测试集上的Loss:1.9342697858810425
整体测试集上的正确率：0.28529998660087585
模型已保存
-----------第2轮训练开始----------
19.929592847824097
训练次数：800，loss:1.8599021434783936
21.560749530792236
训练次数：900，loss:1.830407738685608
23.126902103424072
训练次数：1000，loss:1.8861784934997559
24.68179488182068
训练次数：1100，loss:1.9548356533050537
26.3341965675354
训练次数：1200，loss:1.6886950731277466
27.88399076461792
训练次数：1300，loss:1.6350390911102295
29.45902729034424
训练次数：1400，loss:1.74678373336792
31.034712076187134
训练次数：1500，loss:1.7